# Chapter 1

In [1]:
(ql:quickload "kenzo")

To load "kenzo":
  Load 1 ASDF system:
    kenzo
; Loading "kenzo"



("kenzo")

In [2]:
(cat:cat-init)


---done---

NIL

### 1.2.2 Ordering the generators

In [3]:
(documentation 'cat:f-cmpr 'function)

"--------------------------------------------------------------[function-doc]
F-CMPR
Args: (n1 n2)
Returns :LESS, :EQUAL, or :GREATER, according to the result of the canonical
comparison of the integers N1 and N2.
------------------------------------------------------------------------------"

In [4]:
(cat:f-cmpr 123 789)

:LESS

In [5]:
(documentation 'cat:s-cmpr 'function)

"--------------------------------------------------------------[function-doc]
S-CMPR
Args: (symbol1 symbol2)
Returns :LESS, :EQUAL, or :GREATER, according to the result of the Lisp string
comparison function of the strings (SYMBOL-NAME SYMBOL1) and
(SYMBOL-NAME SYMBOL2).
------------------------------------------------------------------------------"

In [6]:
(cat:s-cmpr 'circulation 'circular)

:GREATER

In [7]:
(documentation 'cat:l-cmpr 'function)

"--------------------------------------------------------------[function-doc]
L-CMPR
Args: (list1 list2)
Returns :LESS, :EQUAL, or :GREATER, according to the lexicographical ordering
of the generator lists LIST1 and LIST2.
------------------------------------------------------------------------------"

In [8]:
(cat:l-cmpr '(1 a b) '(1 a))

:GREATER

### 1.2.3 Functions handling combinations

In [9]:
(documentation 'cat:cmbn 'function)

"--------------------------------------------------------------[function-doc]
CMBN
Args: (degr &rest rest)
Returns a combination of degree DEGR from a sum of terms provided as a sequence
CF1 GNRT1 CF2 GNRT2 ... CFn GNRTn of coefficient / generator pairs in REST.
REST can be of arbitrary length including NIL, in which case the combination
is an instance of the null combination of degree DEGR.
------------------------------------------------------------------------------"

In [10]:
(cat:def comb1 (cat:cmbn 1 1 'u 2 'v 3 'w 4 'z))


----------------------------------------------------------------------{CMBN 1}
<1 * U>
<2 * V>
<3 * W>
<4 * Z>
------------------------------------------------------------------------------


In [11]:
(documentation 'cat:cmbn-non-zero-p 'function)

"-----------------------------------------------------------------[macro-doc]
CMBN-NON-ZERO-P
Args: (cmbn)
Tests if the combination CMBN is a non-null combination of any degree.
------------------------------------------------------------------------------"

In [12]:
(cat:cmbn-non-zero-p comb1)

T

The reader function $\texttt{cat:cmbn-list}$ gets the list of terms of the combination $\texttt{cmbn}$. Beware: a term is not a Kenzo object.

In [13]:
(cat:cmbn-list comb1)

((1 . U) (2 . V) (3 . W) (4 . Z))

In [14]:
(cat:def term3 (third (cat:cmbn-list comb1)))

(3 . W)

In [15]:
(documentation 'cat:cffc 'function)

"-----------------------------------------------------------------[macro-doc]
CFFC
Args: (term)
Returns the the coefficient (integer) of a term.
------------------------------------------------------------------------------"

In [16]:
(cat:cffc term3)

3

In [17]:
(documentation 'cat:gnrt 'function)

"-----------------------------------------------------------------[macro-doc]
GNRT
Args: (term)
Returns the the generator of a term.
------------------------------------------------------------------------------"

In [18]:
(cat:gnrt term3)

W

In [19]:
(documentation 'cat:cmbn-opps 'function)

"--------------------------------------------------------------[function-doc]
CMBN-OPPS
Args: (cmbn)
Returns the combination opposite combination of CMBN.
------------------------------------------------------------------------------"

In [20]:
(cat:def mcomb1 (cat:cmbn-opps comb1))


----------------------------------------------------------------------{CMBN 1}
<-1 * U>
<-2 * V>
<-3 * W>
<-4 * Z>
------------------------------------------------------------------------------


In [21]:
(documentation 'cat:n-cmbn 'function)

"--------------------------------------------------------------[function-doc]
N-CMBN
Args: (n cmbn)
Returns N times the combination CMBN. N must be non-zero.
------------------------------------------------------------------------------"

In [22]:
(cat:def comb2 (cat:n-cmbn 10 comb1))


----------------------------------------------------------------------{CMBN 1}
<10 * U>
<20 * V>
<30 * W>
<40 * Z>
------------------------------------------------------------------------------


In [23]:
(documentation 'cat:2cmbn-add 'function)

"--------------------------------------------------------------[function-doc]
2CMBN-ADD
Args: (cmpr cmbn1 cmbn2)
Returns the combination, which is the sum of CMBN1 and CMBN2. The first
argument, CMPR, must be a function or macro, which is used to compare the
generators of the combination arguments and to order the terms of the result
combination.
------------------------------------------------------------------------------"

In [24]:
(cat:def cmb12 (cat:2cmbn-add #'cat:s-cmpr comb1 comb2))


----------------------------------------------------------------------{CMBN 1}
<11 * U>
<22 * V>
<33 * W>
<44 * Z>
------------------------------------------------------------------------------


In [25]:
(documentation 'cat:2cmbn-sbtr 'function)

"--------------------------------------------------------------[function-doc]
2CMBN-SBTR
Args: (cmpr cmbn1 cmbn2)
Returns the combination, which is the difference of CMBN1 and CMBN2. The first
argument, CMPR, must be a function or macro, which is used to compare the
generators of the combination arguments and to order the terms of the result
combination.
------------------------------------------------------------------------------"

In [26]:
(documentation 'cat:ncmbn-add 'function)

"--------------------------------------------------------------[function-doc]
NCMBN-ADD
Args: (cmpr cmbn &rest rest)
Returns the sum of an arbitrary number of combinations. The first argument,
CMPR, must be a function or macro, which is used to compare the generators of
the input combinations and to order the terms of the result combination.
------------------------------------------------------------------------------"

In [27]:
(cat:ncmbn-add #'cat:s-cmpr
               comb1 comb2 comb1 comb2 comb1 comb2 comb1 comb2 comb1 comb2)


----------------------------------------------------------------------{CMBN 1}
<55 * U>
<110 * V>
<165 * W>
<220 * Z>
------------------------------------------------------------------------------


### 1.3.1 The function build-chcm

#### A first example of a chain complex

In [28]:
(cat:def diabolo-cmpr #'cat:s-cmpr)

#<Compiled-function CAT:S-CMPR #x15064DDE>

In [29]:
(cat:def diabolo-basis #'(lambda (dmn)
                                 (case dmn
                                     (0 '(s0 s1 s2 s3 s4 s5))
                                     (1 '(s01 s02 s12 s23 s34 s35 s45))
                                     (2 '(s345))
                                     (otherwise nil))))

#<Anonymous Function #x1521F1EE>

In [30]:
(cat:def diabolo-bspn 's0)

S0

In [31]:
(cat:def diabolo-pure-dffr
         #'(lambda (dmn gnr)
                   (unless (<= 0 dmn 2)
                       (error "Incorrect dimension for diabolo-dp."))
                   (case dmn
                       (0 (cat:cmbn -1))  ; Note the null combination of degree -1
                       (1 (case gnr
                              (s01 (cat:cmbn 0 -1 's0 1 's1))
                              (s02 (cat:cmbn 0 -1 's0 1 's2))
                              (s12 (cat:cmbn 0 -1 's1 1 's2))
                              (s23 (cat:cmbn 0 -1 's2 1 's3))
                              (s34 (cat:cmbn 0 -1 's3 1 's4))
                              (s35 (cat:cmbn 0 -1 's3 1 's5))
                              (s45 (cat:cmbn 0 -1 's4 1 's5))))
                       (2 (case gnr
                              (s345 (cat:cmbn 1 1 's34 -1 's35 1 's45))))
                       (otherwise (error "Bad generator for complex diabolo")))))

#<Anonymous Function #x15229B46>

In [32]:
(cat:def diabolo-strt :GNRT)

:GNRT

In [33]:
(cat:def diabolo-orgn '(diabolo-for-example))

(DIABOLO-FOR-EXAMPLE)

In [34]:
(documentation 'cat:build-chcm 'function)

"--------------------------------------------------------------[function-doc]
BUILD-CHCM
Args: (&key cmpr basis bsgn intr-dffr strt orgn)
Returns an instance of the class CHAIN-COMPLEX. The keyword arguments are as
follows:

:CMPR CMPR, the comparison function for generators

:BASIS BASIS, the function defining the basis of the freee Z-modules C_p or the
              the keyword :LOCALLY-EFFECTIVE

:BSGN BSGN, a generator, the base point of the underlying set

:INTR-DFFR INTR-DFFR, a Lisp function defining the differential homomorphism
                      for each p (d_p: C_p -> C_{p-1}

:STRT STRT, one of the keywords :GNRT or :CMBN, defining the mapping strategy
            of the differential homomorphism, either by generator or by
            combination. The default is :GNRT

:ORGN ORGN, a list containing a relevant and carefully chosen comment about
            the origin of the chain complex. This comment should be unique
            for a Kenzo session (between calls of CAT-

In [35]:
(cat:def diabolo (cat:build-chcm :cmpr diabolo-cmpr :basis diabolo-basis
                                 :bsgn diabolo-bspn :intr-dffr diabolo-pure-dffr
                                 :strt diabolo-strt :orgn diabolo-orgn))

[K1 Chain-Complex]

In [36]:
(documentation 'cat:chcm 'function)

"--------------------------------------------------------------[function-doc]
CHCM
Args: (idnm)
Return from the list *CHCM-LIST* the chain complex instance with Kenzo
identifier IDNUM or NIL, if it doesn't exist.
------------------------------------------------------------------------------"

In [37]:
(cat:chcm 1)

[K1 Chain-Complex]

In [38]:
(cat:orgn diabolo)

(DIABOLO-FOR-EXAMPLE)

In [39]:
(cat:idnm diabolo)

1

In [40]:
(cat:basis diabolo 0)

(S0 S1 S2 S3 S4 S5)

In [41]:
(cat:basis diabolo 1)

(S01 S02 S12 S23 S34 S35 S45)

In [42]:
(cat:basis diabolo 2)

(S345)

In [43]:
(cat:basis diabolo 10)

NIL

In [44]:
(cat:dffr diabolo 2 's345)


----------------------------------------------------------------------{CMBN 1}
<1 * S34>
<-1 * S35>
<1 * S45>
------------------------------------------------------------------------------


In [45]:
(cat:dffr diabolo (cat:dffr diabolo 2 's345))


----------------------------------------------------------------------{CMBN 0}
------------------------------------------------------------------------------


#### An important trivial case: the unit chain complex, $\mathbb{Z}$

In [46]:
(cat:def ZCC
         (the cat:chain-complex
              (cat:build-chcm
               :cmpr #'(lambda (gnrt1 gnrt2)
                               (declare (ignore gnrt1 gnrt2))
                               (the cat:cmpr :equal))
               :basis #'(lambda (n)
                                (the list
                                     (if (zerop n) '(:Z-gnrt) cat:+empty-list+)))
               :bsgn :Z-gnrt
               :intr-dffr #'(lambda (cmbn)
                                    (the cat:cmbn (cat:zero-cmbn (1- (cat:cmbn-degr cmbn)))))
               :strt :cmbn
               :orgn '(zcc-constant))))

[K3 Chain-Complex]

In [47]:
(documentation 'cat:z-chcm 'function)

"--------------------------------------------------------------[function-doc]
Z-CHCM
Args: ()
Build the unit chain complex.
------------------------------------------------------------------------------"

#### The chain complex $\texttt{circle}$

In [48]:
(defun CIRCLE ()
    (the cat:chain-complex
         (cat:build-chcm
          :cmpr #'(lambda (gnrt1 gnrt2)
                          (declare (ignore gnrt1 gnrt2))
                          (the cat:cmpr :equal))
          :basis #'(lambda (dmns)
                           (the list
                                (case dmns (0 '(*)) (1 '(s1))
                                    (otherwise cat:+empty-list+))))
          :bsgn '*
          :intr-dffr #'cat:zero-intr-dffr
          :strt :cmbn
          :orgn '(circle))))

CIRCLE

## 1.4 Morphisms

In [49]:
(documentation 'cat:morphism 'type)

"----------------------------------------------------------------[class-doc]
MORPHISM
Slots: (sorc trgt degr intr strt ???-clnm ?-clnm rslts idnm orgn)
Intances of this class represent morphisms between chain complexes, and the
differential homomorphism of a chain complex is treated as a morphism of
degree -1. The class has 10 slots:

1. SORC, an object of type CHAIN-COMPLEX, the source chain complex of this
   morphism. Its reader function is SORC.

2. TRGT, an object of type CHAIN-COMPLEX, the target chain complex of this
   morphism. Its reader function is TRGT.

3. DEGR, an integer, the degree of the morphism. Its reader function is DEGR.

4. INTR, a Lisp INTeRnal function implementing the morphism, taking account
   of the strategy STRT. Its reader function is INTR.

5. STRT, a symbol, one of :GNRT or :CMBN. Its reader function is STRT.

   If this slot is :GNRT, the implementation of INTR is a function:
     (function (degr gnrt) cmbn)
   and for any call of this function, the re

### 1.4.1 The function $\texttt{build-mprh}$

In [50]:
(documentation 'cat:build-mrph 'function)

"--------------------------------------------------------------[function-doc]
BUILD-MRPH
Args: (&key sorc trgt degr intr strt orgn)
Returns an instance of the class MORPHISM. The keyword arguments are as
follows:

:SORC SORC, the source object, a CHAIN-COMPLEX type object

:TRGT TRGT, the target object, a CHAIN-COMPLEX type object

:DEGR DEGR, the degree of the morphism, an integer

:INTR INTR, the Lisp function defining the effective mapping

:STRT STRT, one of the keywords :GNRT or :CMBN, defining the mapping strategy
            of the differential homomorphism, either by generator or by
            combination. The default is :GNRT

:ORGN ORGN, a list containing a relevant and carefully chosen comment about
            the origin of the chain complex. This comment should be unique
            for a Kenzo session (between calls of CAT-INIT), as it is used
            for caching purposes.

Use this function instead of creating instances via the standard constructor
MAKE-INSTANCE.
--

#### The differential homomorphism in a chain complex instance

In [51]:
(cat:cat-init)


---done---

NIL

In [52]:
(cat:def ZCC (cat:z-chcm))

[K1 Chain-Complex]

In [53]:
(cat:def zero-morphism (cat:build-mrph :sorc ZCC
                                       :trgt ZCC
                                       :degr -1
                                       :intr #'(lambda (comb)
                                                      (cat:cmbn (1- (cat:degr comb))))
                                       :strt :cmbn
                                       :orgn '(zero morphism on ZCC)))

[K3 Cohomology-Class on K1 of degree 1]

In [54]:
(cat:def id-morphism (cat:build-mrph :sorc ZCC
                                     :trgt ZCC
                                     :degr 0
                                     :intr #'identity
                                     :strt :cmbn
                                     :orgn '(identity morphism on ZCC)))

[K4 Cohomology-Class on K1 of degree 0]

### 1.4.3 Functions defining morphisms

In [55]:
(cat:def ccn-boundary #'(lambda (dgr gnr)
                                (if (evenp (+ dgr gnr))
                                    (cat:cmbn (1- dgr) 1 (- gnr 10))
                                    (cat:cmbn (1- dgr)))))

#<Anonymous Function #x151DDFE6>

In [56]:
(cat:def ccn (cat:build-chcm :cmpr #'cat:f-cmpr
                             :basis #'(lambda (n) (cat:<a-b< (* 10 n) (* 10 (1+ n))))
                             :bsgn 0
                             :intr-dffr ccn-boundary
                             :strt :gnrt
                             :orgn '(ccn)))

[K5 Chain-Complex]

In [57]:
(cat:def upper-shift (cat:build-mrph
                          :sorc ccn :trgt ccn :strt :gnrt :degr +1
                          :intr #'(lambda (d gn) (cat:cmbn (1+ d) 1 (+ gn 10)))
                          :orgn '(ccn shift +10)))

[K7 Morphism (degree 1): K5 -> K5]

In [58]:
(cat:def lower-shift (cat:build-mrph
                          :sorc ccn :trgt ccn :strt :gnrt :degr -1
                          :intr #'(lambda (d gn) (cat:cmbn (1- d) 1 (- gn 10)))
                          :orgn '(ccn shift -10)))

[K8 Morphism (degree -1): K5 -> K5]

In [59]:
(cat:? ccn 2 22)


----------------------------------------------------------------------{CMBN 1}
<1 * 12>
------------------------------------------------------------------------------


In [60]:
(cat:? ccn (cat:? ccn 2 22))


----------------------------------------------------------------------{CMBN 0}
------------------------------------------------------------------------------


In [61]:
(cat:def combn (cat:cmbn 5 1 50 5 55 9 59))


----------------------------------------------------------------------{CMBN 5}
<1 * 50>
<5 * 55>
<9 * 59>
------------------------------------------------------------------------------


In [62]:
(cat:? ccn combn)


----------------------------------------------------------------------{CMBN 4}
<5 * 45>
<9 * 49>
------------------------------------------------------------------------------


In [63]:
(cat:? ccn (cat:? ccn combn))


----------------------------------------------------------------------{CMBN 3}
------------------------------------------------------------------------------


In [64]:
(cat:? upper-shift 0 6)


----------------------------------------------------------------------{CMBN 1}
<1 * 16>
------------------------------------------------------------------------------


In [65]:
(cat:? lower-shift 5 51)


----------------------------------------------------------------------{CMBN 4}
<1 * 41>
------------------------------------------------------------------------------


In [66]:
(cat:? lower-shift (cat:? lower-shift 5 51))


----------------------------------------------------------------------{CMBN 3}
<1 * 31>
------------------------------------------------------------------------------


In [67]:
(cat:def comb1 (cat:cmbn 1 1 10 2 11 3 12 4 13))


----------------------------------------------------------------------{CMBN 1}
<1 * 10>
<2 * 11>
<3 * 12>
<4 * 13>
------------------------------------------------------------------------------


In [68]:
(cat:def identity? (cat:cmps upper-shift lower-shift))

[K9 Morphism (degree 0): K5 -> K5]

In [69]:
(cat:degr identity?)

0

In [70]:
(cat:? identity? comb1)


----------------------------------------------------------------------{CMBN 1}
<1 * 10>
<2 * 11>
<3 * 12>
<4 * 13>
------------------------------------------------------------------------------


In [71]:
(cat:2cmbn-sbtr (cat:cmpr ccn) comb1 (cat:? identity? comb1))


----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------


In [72]:
(cat:def upper-shift2 (cat:cmps upper-shift upper-shift))

[K10 Morphism (degree 2): K5 -> K5]

In [73]:
(cat:degr upper-shift2)

2

In [74]:
(cat:? upper-shift2 comb1)


----------------------------------------------------------------------{CMBN 3}
<1 * 30>
<2 * 31>
<3 * 32>
<4 * 33>
------------------------------------------------------------------------------


In [75]:
(cat:def twice-up-shift (cat:add upper-shift upper-shift))

[K11 Morphism (degree 1): K5 -> K5]

In [76]:
(cat:degr twice-up-shift)

1

In [77]:
(cat:? twice-up-shift comb1)


----------------------------------------------------------------------{CMBN 2}
<2 * 20>
<4 * 21>
<6 * 22>
<8 * 23>
------------------------------------------------------------------------------


In [78]:
(cat:def up-d (cat:cmps upper-shift (cat:dffr1 ccn)))

[K12 Morphism (degree 0): K5 -> K5]

In [79]:
(cat:def d-up (cat:cmps (cat:dffr1 ccn) upper-shift))

[K13 Morphism (degree 0): K5 -> K5]

In [80]:
(cat:? up-d 1 11)


----------------------------------------------------------------------{CMBN 1}
<1 * 11>
------------------------------------------------------------------------------


In [81]:
(cat:? d-up 1 11)


----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------


In [82]:
(cat:def comb3 (cat:cmbn 1 1 10 2 11 3 12 4 13 5 14 6 15))


----------------------------------------------------------------------{CMBN 1}
<1 * 10>
<2 * 11>
<3 * 12>
<4 * 13>
<5 * 14>
<6 * 15>
------------------------------------------------------------------------------


In [83]:
(cat:? up-d comb3)


----------------------------------------------------------------------{CMBN 1}
<2 * 11>
<4 * 13>
<6 * 15>
------------------------------------------------------------------------------


In [84]:
(cat:? d-up comb3)


----------------------------------------------------------------------{CMBN 1}
<1 * 10>
<3 * 12>
<5 * 14>
------------------------------------------------------------------------------


### 1.4.4 Accessing $\texttt{Kenzo}$ objects

In [85]:
(cat:k 1)

[K1 Chain-Complex]

In [86]:
(cat:kd 1)


Object: [K1 Chain-Complex]
  Origin: (Z-CHCM)



NIL

In [87]:
(cat:k 3)

[K3 Cohomology-Class on K1 of degree 1]

In [88]:
(cat:kd 3)


Object: [K3 Cohomology-Class on K1 of degree 1]
  Origin: (ZERO MORPHISM ON ZCC)



NIL

In [89]:
(cat:kd 8)


Object: [K8 Morphism (degree -1): K5 -> K5]
  Origin: (CCN SHIFT -10)



NIL

In [90]:
(cat:kd 5)


Object: [K5 Chain-Complex]
  Origin: (CCN)



NIL

In [91]:
(cat:kd 9)


Object: [K9 Morphism (degree 0): K5 -> K5]
  Origin: (2MRPH-CMPS [K7 Morphism (degree 1): K5 -> K5] [K8 Morphism (degree -1): K5 -> K5] GNRT)



NIL

In [92]:
(cat:kd2 9)


Object: [K9 Morphism (degree 0): K5 -> K5]
  Origin: (2MRPH-CMPS [K7 Morphism (degree 1): K5 -> K5] [K8 Morphism (degree -1): K5 -> K5] GNRT)


Object: [K8 Morphism (degree -1): K5 -> K5]
  Origin: (CCN SHIFT -10)


Object: [K7 Morphism (degree 1): K5 -> K5]
  Origin: (CCN SHIFT 10)



(9 8 7)